## IMPORTS

In [1]:
# FROM SAGE

from sage.rings.polynomial.pbori.pbori import *
from sage.rings.polynomial.pbori import *
from sage.rings.polynomial.pbori.ll import ll_encode
from sage.rings.polynomial.pbori.ll import ll_red_nf_redsb

import time


# FROM LOCAL FILES
#load_attach_path with the correct path
load("WBMULT_AFFMULT.sage")
load("WBMULT_TOOLBOX.sage")
load("WBMULT_CONVERTION.sage")

## PARAMETERS

In [2]:
# Extension Degree
n = 85
# Number of minus equations
a = 0
# Size of packs for minus
n_a = 0
# Number of projected variables
p = 0
# Dimension of Im(Q) for hatplus
t = 6
# Size of packs for hatplus
n_t = 3


In [3]:
# Size of blocs for polynomial bit representation
mbits = 1024
# d_aff is the affine degree of the affine multiple
d_aff = 3
# Number of monomials in y 
Nmon = sigma(n,d_aff)
Nint = int(Nmon/mbits)

## STRUCTURE

In [4]:
# EXTENSION (in z)
# We use minimal weight polynomials to get fast reduction for polynomials later on.
BIG_FIELD.<z> = GF(2**n,modulus='minimal_weight')
# FRACTION RING (in y)
PR = BIG_FIELD['y']
y = PR.gens()[0]
FR = Frac(PR)
# POLYNOMIAL RING (in x)
POLY_RING = PolynomialRing(FR,'x')
x = POLY_RING.gens()[0]

## INTERNAL POLYNOMIAL 

In [5]:
# Declaration of base 
coeff_1  = BIG_FIELD.random_element()
F_poly  = x**3 + coeff_1*x**2

In [6]:
# WITH SOME HATPLUS
Im_Q_base  = [BIG_FIELD.random_element() for i in range(t)]

In [8]:
# CORRECT IT TO COVER Im_Q
F_hatplus = BIG_FIELD(1)
# Draw n_t elt from Im Q
Im_Q = []
Im_Q_coeff_list = []
for i in range(n_t):
    ltemp = []
    Poly_Im_Q = BIG_FIELD(0)
    for j in range(t):
        temp = choice([0,1])
        ltemp.append(temp)
        Poly_Im_Q = Poly_Im_Q + temp*Im_Q_base[j]    
    Im_Q.append(Poly_Im_Q)
    Im_Q_coeff_list.append(ltemp)
for i in range(n_t):
    F_hatplus = F_hatplus * (F_poly + y + Im_Q[i])

In [1]:
#Im_Q_coeff_list
#F_hatplus.monomials()

## SMALL AFFINE MULTIPLE

In [11]:
MON_RED  = mons_reduction(F_hatplus ,POLY_RING)
# COMPUTING COEFFS in BIG_FIELD(y)
Y_AFF = affine_coeffs(MON_RED)

In [12]:
#Y_AFF

In [13]:
# Gathering degrees in Y
Y_DEGREES = set([])
for i in range(len(Y_AFF)):
    temp = PR(Y_AFF[i]).monomials()
    l_temp = []
    for j in range(len(temp)):
        l_temp.append(temp[j].degree())
    Y_DEGREES = Y_DEGREES.union(set(l_temp))

Y_DEGREES = list((Y_DEGREES))
Y_DEGREES.sort()        
        

In [2]:
#Y_DEGREES

## STRUCTURE

In [15]:
# We use minimal weight polynomials to get fast reduction for polynomials.
#F.<xx> = GF(2**n, modulus='minimal_weight')
MODULUS = BIG_FIELD.modulus()
PTI_MULT = declare_ring([Block('X',n),Block('Y',n)],globals())
PTIx_MULT = declare_ring([Block('X',n)],globals())
PTIy_MULT = declare_ring([Block('Y',n)],globals())
GROS_MULT.<Z> = PolynomialRing(PTI_MULT)
R.<X>  = PolynomialRing(BIG_FIELD)
MODULUS_inj = MODULUS(Z)
parsing_RING= PolynomialRing(GF(2),'zz',2*n)


## COMPOSITION WITH S AND T

In [16]:
## PREPARING AFFINE TRANSFORMATIONS

In [17]:
## Sampling S and T, linear secrets in GL(n,GF(2))
MG = GL(n,GF(2))
#Sampling S in GL and its affine vector S_v
S = MG.random_element()
S_v = BIG_FIELD.random_element()
S_v_coord = conv_GF_to_VS(n,S_v,BIG_FIELD)
#Sampling T in GL an its affine vector T_v
T = MG.random_element()
T_v = BIG_FIELD.random_element()
T_v_coord = conv_GF_to_VS(n,T_v,BIG_FIELD)
# For Tinv, we have to add T_v then inversing T
Tinv = T.inverse()

In [18]:
# We need S_poly as input for the secret key and private key
S_poly = AFF_to_PTI(n,list(PTI_MULT.gens()),S,S_v_coord)
# We need Tinv as input for the affine multiple
# Adding T_v, then inversing the linear part
Tinv_input = [PTI_MULT.gens()[i+n]+T_v_coord[i] for i in range(n)]
Tinv_poly = AFF_to_PTI(n,Tinv_input,Tinv,[0]*n)
# We parse T to compute the public key
T_parsed = parse_GL(n,T)

 ## COMPUTING THE PROJECTION OF THE AFFINE MULTIPLE

### Computing polynomials in X and Y

In [19]:
# Gathering the monomials in x in the vector space.
startT = time.time() 
X_MON = [1,]
for i in range (len(Y_AFF)):
    X_MON.append(compute_monomial_PTIlist_GROS_ctrl_size(n,2**i,MODULUS,S_poly,GROS_MULT))
        
print(str(time.time()-startT))

7.31457257270813


In [20]:
# Gathering the monomials in y in the vector space.
# It takes some time 
startT = time.time() 
Y_MON = [1,]
for i in Y_DEGREES[1:]:
    Y_MON.append(compute_monomial_PTIlist_GROS_ctrl_size(n,i,MODULUS,Tinv_poly,GROS_MULT))    
print(str(time.time()-startT))

100.56440854072571


In [21]:
# Gathering the coefficients
startT = time.time() 
COEFF_LIST = []
for i in Y_AFF:
    # Constant terms don't have coeffs.
    if i != 0 and i != 1 :
        L = PR(i).coefficients(sparse = false)
        COEFFs = []
        # We can evaluate constants however.
        for j in L :
            coeff = conv_GF_to_VS(n,BIG_FIELD(j),BIG_FIELD)
            coeff =  sum([coeff[k]*(GROS_MULT.gens()[0])^k for k in range(len(coeff))])
            # We convert them into vector space format
            COEFFs.append(coeff)
        COEFF_LIST.append(COEFFs)
    else:
        COEFF_LIST.append([i])
    
print(str(time.time()-startT))

0.2550232410430908


In [22]:
# Computing polynomials in Y
# It takes some time
startT = time.time() 
AFF_MULTy = []
for i in range(len(Y_AFF)):
    TEMP = 0
    for j in range(len(COEFF_LIST[i])):
        if COEFF_LIST[i][j] != 0 :
            temp = (COEFF_LIST[i][j]* Y_MON[Y_DEGREES.index(j)])%MODULUS_inj
            TEMP = TEMP + temp
    AFF_MULTy.append(TEMP)
        
print(str(time.time()-startT))

146.40413999557495


In [23]:
startT = time.time() 

# initialisation of X vector
Xp = 0
for i in range(n):
        Xp = PTI_MULT.gens()[i]* (GROS_MULT.gens()[0]^i) + Xp
        
        
# initialisation of Y vector
Yp = 0
for i in range(n):
        Yp = PTI_MULT.gens()[i+n]* (GROS_MULT.gens()[0]^i) + Yp

    
# Computing the product functions
prod_XY = []
for i in X_MON:
    prod_XY.append((Yp*i)%MODULUS_inj)
        
print(str(time.time()-startT))

12.389324426651001


In [ ]:
## We work with list of integers instead of polynomials for the rest of the algorithms to speed up the computations.

In [27]:
# Transforming elements of AFF_MULTy in intlist format
MON3 = mon3(n)
# It takes some time
startT = time.time() 
AFF_MULTy_intlist = []
for i in range(len(AFF_MULTy)):
    L = []
    if AFF_MULTy[i] != 0:
        for j in range(n):
            if AFF_MULTy[i][j] != 0 :
                L.append(poly_to_intlist(n,mbits,AFF_MULTy[i][j],MON3))
            else:
                #L.append([0]*Nmon)
                L.append([0]*(int(Nmon//mbits)+(Nmon%mbits != 0)))
        AFF_MULTy_intlist.append(L)
    else:
        for j in range(n):
            #L.append([0]*Nmon)
            L.append([0]*(int(Nmon//mbits)+(Nmon%mbits != 0)))
        AFF_MULTy_intlist.append(L)
print(str(time.time()-startT)) 

76.38515090942383


In [28]:
#AFF_MULTy_intlist

In [29]:
# INPUT : integer n , prod_list are the coordinates of the multiplication in GF(2*n)
# OUTPUT : XY_list,  XY_list[i][j] is the variable that is multiple of x_j in the ith coordinate of prod_list

# can maybe optimize the 'getting variables part'
def prod_GF_to_XYlist(n,prod_list):
    XY_list = []
    # going through coordinates
    for i in range(n):
        line = []
        for j in range(n):
            # getting the multiples of variable j
            m = prod_list[i].set().multiples_of(PTI_MULT.gens()[j].lm())
            # evaluating these multiples at 1 in j
            m = m.subset1(j)
            # getting the variables left
            var_list = list(m.set().vars().iterindex())
            if var_list == []:
                line.append([])
            else:
                line.append(var_list)
        # getting the constant coefficient by evaluating to 0
        dictio = {}
        for k in range(n):
            dictio[PTI_MULT.gens()[k]] = 0
        temp = prod_list[i].subs(dictio)
        #### The last term is the constant coefficient ####
        line.append(list(temp.set().vars().iterindex())+[temp.constant_coefficient()])
        XY_list.append(line)
    return(XY_list)
            

In [30]:
startT = time.time() 
AFF_MULT_intlist = [[[0 for k in range(Nint +1)] for i in range(n+1)]for j in range(n)]
# Going through AFF_MULTy_intlist
for i in range(len(AFF_MULTy_intlist)):
    # we build the addition matrix
    matoche  = prod_GF_to_XYlist(n,prod_XY[i].coefficients(sparse = false))
    # we add the polynomials, going trough the matoche
    #lines
    for j in range(n):
        for k in range(n):
            for ind in matoche[j][k]:
                AFF_MULT_intlist[j][k] = xor_intlist(AFF_MULT_intlist[j][k],AFF_MULTy_intlist[i][ind-n])
        # Adding the constant term
        #print(matoche[j][n-1][:-1])
        for ind in matoche[j][n][:-1]:
             AFF_MULT_intlist[j][n] = xor_intlist(AFF_MULT_intlist[j][n],AFF_MULTy_intlist[i][ind-n])
      
        AFF_MULT_intlist[j][n][Nint] = \
        AFF_MULT_intlist[j][n][Nint].__xor__(int(matoche[j][n][-1])<< (Nint%mbits))
            
print(str(time.time()-startT))  
            

99.41975474357605


# PUBLIC KEY

In [31]:
## PUBLIC KEY over GF(2)^n

In [39]:
# COMPUTES THE PUBLIC KEY 
COEFF_1_vs  = conv_GF_to_VS(n,coeff_1,BIG_FIELD)
COEFF_1_vs =  sum([COEFF_1_vs[k]*(GROS_MULT.gens()[0])^k for k in range(len(COEFF_1_vs))])

# Generating  t random's quadratic
t_QUADS = [PTIx_MULT.random_element(degree=2,terms = int(Nmon/2)) for i in range(t)]

# COMPUTING Q
Q = 0
for i in range(t):
    temp_Q = conv_GF_to_VS(n,Im_Q_base[i],BIG_FIELD)
    temp_Q = sum([temp_Q[k]*t_QUADS[i]*(GROS_MULT.gens()[0])^k for k in range(len(temp_Q))])
    Q = Q + temp_Q


In [40]:
PLOP1 = (X_MON[1]*X_MON[2])%MODULUS_inj
PLOP2 = (COEFF_1_vs*X_MON[2])%MODULUS_inj
PLOP3 = (PLOP1+PLOP2+Q)

In [41]:
PK  = PLOP3.coefficients(sparse = false)
PUBLIC_KEY = [compo_GL(T_parsed[i],PK)+T_v_coord[i] for i in range(n)]

In [42]:
## VERIF WITH PUBLIC KEY

In [43]:
test_sign = BIG_FIELD.random_element()
test_sign_coord = conv_GF_to_VS(n,test_sign,BIG_FIELD)
S_test_sign_coord = list(S*vector(test_sign_coord) + vector(S_v))

In [44]:
# Evaluating Quads
eval_QUADS = []
for i in range(t):
    eval_QUADS.append(eval_poly_set(test_sign_coord, t_QUADS[i].set(),PTI_MULT))
    
# IM Q
Im_Q_eval  = 0
for i in range(t):
    Im_Q_eval = Im_Q_eval + Im_Q_base[i]*BIG_FIELD(eval_QUADS[i])
    

In [45]:
S_test_sign = conv_VS_to_GF(S_test_sign_coord,BIG_FIELD)
test_msg = S_test_sign**3 + coeff_1 * S_test_sign**2 + Im_Q_eval
test_msg = conv_GF_to_VS(n,test_msg,BIG_FIELD)
test_msg_coord = list(T*vector(test_msg) + vector(T_v))


In [46]:
# CHECK FOR CORRECTNESS
pot_msg = []
for i in range(n):
    pot_msg.append(eval_poly_set(test_sign_coord, PUBLIC_KEY[i].set(),PTI_MULT))
print(test_msg_coord == pot_msg)

True


In [47]:
# CHECK FOR IMQ in AFFMULT, if FALSE, it won't sign.
while not(eval_QUADS in Im_Q_coeff_list):
    test_sign = BIG_FIELD.random_element()
    test_sign_coord = conv_GF_to_VS(n,test_sign,BIG_FIELD)
    eval_QUADS = []
    for i in range(t):
        eval_QUADS.append(eval_poly_set(test_sign_coord, t_QUADS[i].set(),PTI_MULT))

test_msg_coord = []
for i in range(n):
    test_msg_coord.append(eval_poly_set(test_sign_coord, PUBLIC_KEY[i].set(),PTI_MULT))

In [48]:
# CONVERTING THE PK INTO INTLIST

In [49]:
PK_MON = pk_mon(n)

In [50]:
startT = time.time() 
PK_intlist = []
for i in range(n):
    if PUBLIC_KEY[i] != 0:
            PK_intlist.append(pk_to_intlist(n,mbits,PUBLIC_KEY[i],PK_MON))
    else:    
            PK_intlist.append([0]*(int(sigma(n,2)//mbits)+(sigma(n,2)%mbits != 0)))
print(str(time.time()-startT)) 

1.259784460067749


In [51]:
# CHECK ON THE CONVERTION
test_sign_coord = [int(i) for i in test_sign_coord]
MON_SIGN = sign_to_monintlist(n,mbits,test_sign_coord)
CONV_MSG = []
for i in range(n):
    CONV_MSG.append(GF(2)(eval_intlist_poly(PK_intlist[i],MON_SIGN)))
CONV_MSG == test_msg_coord

True

## SIGNING

In [52]:
# Computing monomials in the msg
test_msg_coord = [int(i) for i in test_msg_coord]
MON_MSG = msg_to_monintlist(n,mbits,test_msg_coord)

In [53]:
# Building matrix
MAT = []
for i in range(n):
    L = []
    # eval non proj vars
    for j in range(n):
        L.append(GF(2)(eval_intlist_poly(AFF_MULT_intlist[i][j],MON_MSG)))
    # eval cst
    L.append(GF(2)(eval_intlist_poly(AFF_MULT_intlist[i][n],MON_MSG)))
    MAT.append(L)

In [54]:
# KERNEL OF THE LINEAR SYSTEM

In [55]:
sol_space = matrix(MAT).right_kernel()

In [56]:
sol_space

Vector space of degree 86 and dimension 5 over Finite Field of size 2
Basis matrix:
5 x 86 dense matrix over Finite Field of size 2

In [57]:
# CHECK THE SOLUTIONS WITH PK

In [58]:
vector(test_sign_coord + [1]) in sol_space

True

# STORING IN FILES

In [61]:
# assume 8 | mbits
def mbits_list_to_8bits_list(L,mbits):
    Lconv = []
    d = mbits // 8
    for i in range(len(L)-1):
        int_temp = 0
        for j in range(d):
            int_temp = int(sum( [ ((L[i]>>(m + j*8))%2)<<m  for m in range(8)]))
            Lconv.append(int_temp)
    # last mbitd intint
    r = Nmon%mbits
    qd = r//8
    rd = r%8
    for i in range(qd):
        int_temp =  int(sum( [ ((L[len(L)-1]>>(m + i*8))%2)<<m  for m in range(8)]))
        Lconv.append(int_temp)
    # last 8 bit int
    int_temp = int(sum( [ ((L[len(L)-1]>>(m + qd*8))%2)<<m  for m in range(rd)]))
    Lconv.append(int_temp)
    return(Lconv)

In [62]:
# Comverting the polynomials into 8bit lists
startT = time.time()
AFF_MULT_8bit = []
for i in range(n):
    LINE = []
    for j in range(n+1):
        LINE.append(mbits_list_to_8bits_list(AFF_MULT_intlist[i][j],mbits))
    AFF_MULT_8bit.append(LINE)        
    
PK_8bit = []
for i in range(n):
        PK_8bit.append(mbits_list_to_8bits_list(PK_intlist[i],mbits))    
print(str(time.time()-startT))

828.330308675766


In [63]:
def matrix_to_file(n,L,file_name):
    file = open(file_name, "wb")
    for i in range(n):
        for j in range(n+1):
                file.write(bytearray(L[i][j]))
    file.close()               
    return()

# We only give the first n-a coordinates
a = 5
def pk_to_file(n,L,file_name):
    file = open(file_name, "wb")
    for i in range(len(L)-5):
                file.write(bytearray(L[i]))
    file.close()               
    return()

def test_to_file(ntest,T,file_name):
    file = open(file_name, "wb")
    for i in range(ntest):
            file.write(bytearray(T[0][i]))
    for i in range(ntest):
            file.write(bytearray(T[1][i]))
    file.close() 
    
    

In [64]:
startT = time.time()
pk_to_file(n,PK_8bit,'PK_challenge.bin')
matrix_to_file(n,AFF_MULT_8bit,'AFFMULT_challenge.bin')
test_to_file(1,[[test_msg_coord],[test_sign_coord]],'TEST_challenge.bin')
print(str(time.time()-startT))

0.0007603168487548828
